In [3]:
import pandas as pd
import numpy as np
import lightgbm as gbm
from lightgbm import LGBMClassifier
from sklearn import tree

train = pd.read_csv(r'C:\Users\natha\Desktop\Data\train.csv')
test = pd.read_csv(r'C:\Users\natha\Desktop\Data\test.csv')

train_copy = train
test_copy = test
train_id = train["id"]
train_label = train["target"]
test_id = test["id"]

### variables we need to get rid of when training

drop_features = ['id','target']

### Drop the 'calc' features which are apparently useless as said on the competition forum

for col in train_copy.columns:
    if 'calc' in col:
        del train_copy[col]
    
for col in test_copy.columns:
    if 'calc' in col:
        del test_copy[col] 
        
X_train = train_copy.drop(drop_features,axis=1)
y_train = train_copy['target']

feature_names = X_train.columns.tolist()

cat_features = [c for c in feature_names if ('cat' in c and 'count' not in c)]

In [4]:
# Here we dummify the 'cat' features to make them usable for supervised learning

new_set = pd.get_dummies(columns=cat_features, data=train)

# Create new variables based on the dummified 

X_new = new_set.drop(drop_features,axis=1)
y_new = new_set['target']

In [5]:
from sklearn import tree
test_tree = tree.DecisionTreeClassifier(max_depth = 10, min_samples_split = 5, random_state = 42,class_weight='balanced')
test_tree = test_tree.fit(X_new,y_new)
print(test_tree.feature_importances_)
print(test_tree.score(X_new,y_new))

[1.85589894e-02 4.45470339e-02 2.37288528e-02 7.96247283e-03
 8.57996545e-03 6.52890778e-03 0.00000000e+00 4.71753632e-04
 8.14483869e-04 0.00000000e+00 0.00000000e+00 4.71971907e-02
 3.46904450e-03 6.05123134e-02 2.48893572e-03 3.04822391e-02
 2.71508145e-02 1.14765395e-01 3.87504201e-03 1.48812368e-02
 2.50678395e-01 2.69366015e-02 1.73689379e-02 0.00000000e+00
 0.00000000e+00 5.26077669e-03 2.43652215e-03 6.34951638e-04
 9.94046461e-04 3.54412562e-03 2.44562683e-03 0.00000000e+00
 9.42685872e-02 1.48060545e-03 8.16937671e-04 3.30084848e-03
 9.13486921e-04 8.68849581e-04 2.75781152e-04 0.00000000e+00
 8.34326585e-04 1.61065858e-03 0.00000000e+00 3.14410119e-04
 0.00000000e+00 2.30944355e-03 5.34052493e-03 1.02368759e-02
 2.42001157e-03 4.11830161e-03 3.98118604e-04 1.54330915e-03
 0.00000000e+00 5.88773585e-04 7.66365719e-04 9.05520941e-03
 3.40970016e-04 5.87353371e-03 0.00000000e+00 3.43272040e-04
 7.22623861e-03 0.00000000e+00 0.00000000e+00 0.00000000e+00
 4.53280684e-04 0.000000

In [6]:
# Same as above since I forgot to dummify both sets in one go

for col in test_copy.columns:
    if 'calc' in col:
        del test_copy[col]  

new_test = pd.get_dummies(columns=cat_features, data=test_copy)

X_new_test = new_test.drop('id',axis=1)
y_pred = test_tree.predict_proba(X_new_test)[:,1]

In [7]:
gbmc = LGBMClassifier(num_leaves=39,learning_rate=0.01, n_estimators=1211,min_child_weight=7,subsample=0.85,xgboost_dart_mode=True,reg_alpha=1,min_data_in_leaf=1250,bagging_fraction=0.75)
gbmc.fit(X_new,y_new)
pred = gbmc.predict_proba(X_new_test)[:,1]

In [8]:
output = pd.DataFrame( { 'id': test_id , 'target': pred} )
output = output[['id', 'target']]
output.to_csv("LightGBM-v13-last5.csv",index=False)

https://www.kaggle.com/manigandanv/lightgbm-regressor-classifier-cross-validation
http://testlightgbm.readthedocs.io/en/latest/python/lightgbm.html
https://medium.com/@pushkarmandot/https-medium-com-pushkarmandot-what-is-lightgbm-how-to-implement-it-how-to-fine-tune-the-parameters-60347819b7fc
https://github.com/Microsoft/LightGBM/blob/master/docs/Parameters-Tuning.rst
http://lightgbm.readthedocs.io/en/latest/Parameters.html
https://media.readthedocs.org/pdf/lightgbm/latest/lightgbm.pdf